In [1]:
!pip install openai gradio PyPDF2
!pip install openai==0.28
!pip install flask openai
!pip install gradio
!pip install python-docx
!pip install beautifulsoup4
!pip install selenium webdriver-manager

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggi

In [2]:
import PyPDF2
import openai
import gradio as gr
import os
import pandas as pd
import re
from google.colab import files

In [3]:
openai.api_key = "sk-proj-sfbgnhmhvcdvbcnzkS1Hvo2HaaslcAcqOQ8PF_9F78JwlZlyDYA"  #OpenAI API key

In [ ]:
import PyPDF2
import openai
import gradio as gr
import os
import pandas as pd
import re
from google.colab import files  # Для скачивания файлов
import random
import pandas as pd
import os

# Функция для записи результатов в CSV
def save_to_csv(data, file_path):
    try:
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            df = df.append(data, ignore_index=True)
        else:
            df = pd.DataFrame([data])
        df.to_csv(file_path, index=False)
    except Exception as e:
        print(f"Ошибка при сохранении данных в CSV: {e}")


# Функция для извлечения текста из PDF
def extract_text_from_pdf(pdf_file):
    text = ""
    try:
        with open(pdf_file, "rb") as file:
            reader = PyPDF2.PdfReader(file)
            for page in reader.pages:
                text += page.extract_text() + "\n"
    except Exception as e:
        return f"Ошибка при извлечении текста из PDF: {str(e)}"
    return text

# Функция для извлечения контактов (email, телефон) из текста резюме
def extract_contacts(resume_text):
    # Регулярные выражения для поиска email и телефона
    email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
    phone_pattern = r'\+?\d[\d\s\-\(\)]{7,15}\d'

    emails = re.findall(email_pattern, resume_text)
    phones = re.findall(phone_pattern, resume_text)

    # Если нашли email, берем первый
    contact_info = {
        "email": emails[0] if emails else "Не указан",
        "phone": phones[0] if phones else "Не указан"
    }

    return contact_info

# Функция для анализа вакансии (извлечение требований)
def analyze_job_vacancy(job_description_text):
    prompt = f"Проанализируй описание вакансии и извлеки следующие данные:\n1. Основные требования (например, опыт, навыки и образование)\n2. Должностные обязанности\n\nОписание вакансии:\n{job_description_text}"
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "system", "content": "Вы — ассистент, который анализирует вакансии и извлекает требования, опыт и другие детали."},
                      {"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Ошибка при обработке вакансии: {e}"

# Функция для анализа резюме
def analyze_resume_against_vacancy(resume_text, job_requirements):
    prompt = f"Прочитай следующее резюме:\n{resume_text}\n\nСравни с требованиями вакансии:\n{job_requirements}\nОпредели, насколько резюме соответствует указанным критериям, и предоставь детальное сравнение, включая сильные и слабые стороны кандидата."
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "system", "content": "Вы — ассистент, который отбирает кандидатов на вакансии."},
                      {"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Ошибка при анализе резюме: {str(e)}"

# Функция для оценки статуса кандидата
def categorize_candidate(analysis):
    analysis_lower = analysis.lower()
    ideal_keywords = [
        "полностью соответствует",
        "идеально подходит",
        "соответствует всем требованиям",
        "все требования выполнены",
        "кандидат полностью подходит",
        "не выявлено недостатков",
        "отлично соответствует"
    ]

    partial_keywords = [
        "частично соответствует",
        "подходит по некоторым пунктам",
        "выполнены некоторые требования",
        "соответствует большинству требований",
        "имеет соответствие",
        "удовлетворяет ключевым аспектам",
        "есть соответствие по ряду требований",
        "частично подходит"
    ]

    not_ideal_keywords = [
        "не соответствует",
        "не подходит",
        "существенные недостатки",
        "не выполнены основные требования",
        "не удовлетворяет критериям"
    ]

    if any(keyword in analysis_lower for keyword in ideal_keywords):
        return "Идеально подходит"

    elif any(keyword in analysis_lower for keyword in partial_keywords):
        return "Частично подходит"

    elif any(keyword in analysis_lower for keyword in not_ideal_keywords):
        return "Не подходит"

    elif any(keyword in analysis_lower for keyword in ["соответствует", "удовлетворяет", "сильные стороны", "положительные качества"]):
        return "Частично подходит"

    return "Не подходит"

# Функция для работы с интерфейсом User (проверка соответствия)
def assistant_interface_user(job_description, resume_file):
    resume_text = extract_text_from_pdf(resume_file.name)
    job_requirements = analyze_job_vacancy(job_description)
    analysis = analyze_resume_against_vacancy(resume_text, job_requirements)

    candidate_status = categorize_candidate(analysis)

    if candidate_status != "Не подходит":
        return "Вы успешно прошли на второй этап", True  # Возвращаем статус и флаг успешного прохождения
    else:
        return "К сожалению, ваше резюме не соответствует требованиям для первого этапа", False

# Функция для предоставления задания кандидату
def give_assignment_for_candidate(vacancy_description):
    prompt = f"Предоставь задание для кандидата, исходя из следующего описания вакансии: {vacancy_description}\nЗадание должно быть практическим и связано с необходимыми навыками для данной работы."
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "system", "content": "Вы ассистент, который генерирует задания для кандидатов по описанию вакансии."},
                      {"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Ошибка при создании задания: {e}"


# Функция для обработки регистрации на собеседование
def register_interview(name, phone, email, interview_time):
    # Сохраняем информацию о записи в CSV
    new_entry = {
        "Имя": name,
        "Телефон": phone,
        "Email": email,
        "Время собеседования": interview_time
    }

    try:
        df = pd.read_csv("/content/interview_schedule.csv")
        df = df.append(new_entry, ignore_index=True)
    except FileNotFoundError:
        df = pd.DataFrame([new_entry])

    df.to_csv("/content/interview_schedule.csv", index=False)

# Интерфейс Gradio
def download_csv():
    return "/content/candidate_results.csv"



def extract_text_from_pdf2(pdf_file):
    text = ""
    try:
        with open(pdf_file, "rb") as file:
            reader = PyPDF2.PdfReader(file)
            for page in reader.pages:
                text += page.extract_text() + "\n"
    except Exception as e:
        return f"Ошибка при извлечении текста из PDF: {str(e)}"
    return text

# Функция для анализа вакансии (извлечение требований)
def analyze_job_vacancy2(job_description_text):
    prompt = f"Проанализируй описание вакансии и извлеки следующие данные:\n1. Основные требования (например, опыт, навыки и образование)\n2. Должностные обязанности\n\nОписание вакансии:\n{job_description_text}"
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "system", "content": "Вы — ассистент, который анализирует вакансии и извлекает требования, опыт и другие детали."},
                      {"role": "user", "content": prompt}]
        )
        print(f"Ответ OpenAI по вакансии: {response.choices[0].message.content}")  # Отладочный вывод
        return response.choices[0].message.content
    except Exception as e:
        return f"Ошибка при обработке вакансии: {e}"

# Функция для анализа резюме
def analyze_resume_against_vacancy2(resume_text, job_requirements):
    prompt = f"Прочитай следующее резюме:\n{resume_text}\n\nСравни с требованиями вакансии:\n{job_requirements}\nОпредели, насколько резюме соответствует указанным критериям, и предоставь детальное сравнение, включая сильные и слабые стороны кандидата."
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "system", "content": "Вы — ассистент, который отбирает кандидатов на вакансии."},
                      {"role": "user", "content": prompt}]
        )
        print(f"Ответ OpenAI по резюме: {response.choices[0].message.content}")  # Отладочный вывод
        return response.choices[0].message.content
    except Exception as e:
        return f"Ошибка при анализе резюме: {str(e)}"

# Функция для работы с интерфейсом User (проверка соответствия и второй этап)
def assistant_interface_user2(job_description, resume_file):
    resume_text = extract_text_from_pdf(resume_file.name)
    job_requirements = analyze_job_vacancy(job_description)
    analysis = analyze_resume_against_vacancy(resume_text, job_requirements)

    candidate_status = categorize_candidate(analysis)

    print(f"Статус кандидата: {candidate_status}")  # Отладочный вывод

    if candidate_status != "Не подходит":
        # Генерация задачи для второго этапа через OpenAI, если резюме подходит
        task_prompt = f"На основе описания вакансии: {job_description}\nСоздайте задачу для второго этапа собеседования, связанную с этой вакансией. Несложную"
        try:
            task_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "system", "content": "Вы — ассистент, который генерирует задания для собеседований."},
                          {"role": "user", "content": task_prompt}]
            )
            task_description = task_response.choices[0].message.content
            return f"Вы успешно прошли на первый этап. Задание для второго этапа: {task_description}", task_description
        except Exception as e:
            return f"Ошибка при генерации задания для второго этапа: {str(e)}", None
    else:
        return "К сожалению, ваше резюме не соответствует требованиям для первого этапа", None


# Функция для проверки выполнения задания
def check_second_stage_answer2(task_description, user_answer):
    prompt = f"Задание:\n{task_description}\n\nОтвет пользователя:\n{user_answer}\n\nПроверьте, соответствует ли ответ требованиям задания, и дайте краткую оценку, включая правильность выполнения."
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "system", "content": "Вы — ассистент, который проверяет правильность выполнения заданий."},
                      {"role": "user", "content": prompt}]
        )
        evaluation = response.choices[0].message.content

        print(f"Оценка выполнения задания: {evaluation}")  # Отладочный вывод

        # Оценка выполнения задания
        test_result = "Не прошел"
        if "правильно" in evaluation.lower() or "удовлетворительно" in evaluation.lower() or "Задание выполнено верно" in evaluation.lower() or "полностью соответствует требованиям" in evaluation.lower():
            test_result = "Пройден"

        # Формируем данные для записи в CSV
        result_data = {
            "Задание второго этапа": task_description,
            "Ответ пользователя": user_answer,
            "Оценка выполнения": evaluation,
            "Результат теста": test_result
        }

        # Сохраняем результат в CSV
        save_to_csv(result_data, "/content/second_stage_results.csv")

        if test_result == "Пройден":
            return f"Ваш ответ проверен. Вы приглашены на собеседование! Оценка: {evaluation}"
        else:
            return f"Ваш ответ проверен. К сожалению, вы не прошли дальше. Оценка: {evaluation}"
    except Exception as e:
        return f"Ошибка при проверке задания: {str(e)}"



with gr.Blocks() as iface:
    with gr.TabItem("Результаты анализа HR"):
        gr.Markdown("### Результаты анализа резюме")
        job_description = gr.Textbox(label="Описание вакансии", lines=5, placeholder="Введите описание вакансии...")
        resume_files = gr.File(label="Загрузите резюме (PDF)", file_count="multiple", type="filepath")
        output_results = gr.DataFrame(label="Результаты анализа")

        submit_results = gr.Button("Получить результаты")
        submit_results.click(assistant_interface_results, [job_description, resume_files], output_results)

        download_button = gr.Button("Скачать результаты")
        download_button.click(download_csv, [], None)  # Для скачивания CSV

    with gr.TabItem("Проверка для пользователя"):
        gr.Markdown("### Проверка соответствия резюме вакансии")
        job_description_user2 = gr.Textbox(label="Описание вакансии", lines=5, placeholder="Введите описание вакансии...")
        resume_file_user2 = gr.File(label="Загрузите ваше резюме (PDF)", type="filepath")
        output_user2 = gr.Textbox(label="Результат проверки")
        task_output2 = gr.Textbox(label="Задание для второго этапа", interactive=False)

        submit_user2 = gr.Button("Проверить соответствие")
        submit_user2.click(assistant_interface_user2, [job_description_user2, resume_file_user2], [output_user2, task_output2])

        gr.Markdown("### Ответ на задание второго этапа")
        user_task_response2 = gr.Textbox(label="Ваш ответ на задание", lines=5, placeholder="Введите ваш ответ...")
        output_second_stage2 = gr.Textbox(label="Результат проверки")

        submit_task2 = gr.Button("Отправить ответ")
        submit_task2.click(check_second_stage_answer2, [task_output2, user_task_response2], output_second_stage2)


iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://47749d3194066cf208.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [5]:
import PyPDF2
import openai
import gradio as gr
import os
import pandas as pd
import re
from google.colab import files  # Для скачивания файлов
import random
import pandas as pd
import os

# Функция для записи результатов в CSV
def save_to_csv(data, file_path):
    try:
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            df = df.append(data, ignore_index=True)
        else:
            df = pd.DataFrame([data])
        df.to_csv(file_path, index=False)
    except Exception as e:
        print(f"Ошибка при сохранении данных в CSV: {e}")


# Функция для извлечения текста из PDF
def extract_text_from_pdf(pdf_file):
    text = ""
    try:
        with open(pdf_file, "rb") as file:
            reader = PyPDF2.PdfReader(file)
            for page in reader.pages:
                text += page.extract_text() + "\n"
    except Exception as e:
        return f"Ошибка при извлечении текста из PDF: {str(e)}"
    return text

# Функция для извлечения контактов (email, телефон) из текста резюме
def extract_contacts(resume_text):
    # Регулярные выражения для поиска email и телефона
    email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
    phone_pattern = r'\+?\d[\d\s\-\(\)]{7,15}\d'

    emails = re.findall(email_pattern, resume_text)
    phones = re.findall(phone_pattern, resume_text)

    # Если нашли email, берем первый
    contact_info = {
        "email": emails[0] if emails else "Не указан",
        "phone": phones[0] if phones else "Не указан"
    }

    return contact_info

# Функция для анализа вакансии (извлечение требований)
def analyze_job_vacancy(job_description_text):
    prompt = f"Проанализируй описание вакансии и извлеки следующие данные:\n1. Основные требования (например, опыт, навыки и образование)\n2. Должностные обязанности\n\nОписание вакансии:\n{job_description_text}"
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "system", "content": "Вы — ассистент, который анализирует вакансии и извлекает требования, опыт и другие детали."},
                      {"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Ошибка при обработке вакансии: {e}"

# Функция для анализа резюме
def analyze_resume_against_vacancy(resume_text, job_requirements):
    prompt = f"Прочитай следующее резюме:\n{resume_text}\n\nСравни с требованиями вакансии:\n{job_requirements}\nОпредели, насколько резюме соответствует указанным критериям, и предоставь детальное сравнение, включая сильные и слабые стороны кандидата."
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "system", "content": "Вы — ассистент, который отбирает кандидатов на вакансии."},
                      {"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Ошибка при анализе резюме: {str(e)}"

# Функция для оценки статуса кандидата
def categorize_candidate(analysis):
    analysis_lower = analysis.lower()
    ideal_keywords = [
        "полностью соответствует",
        "идеально подходит",
        "соответствует всем требованиям",
        "все требования выполнены",
        "кандидат полностью подходит",
        "не выявлено недостатков",
        "отлично соответствует"
    ]

    partial_keywords = [
        "частично соответствует",
        "подходит по некоторым пунктам",
        "выполнены некоторые требования",
        "соответствует большинству требований",
        "имеет соответствие",
        "удовлетворяет ключевым аспектам",
        "есть соответствие по ряду требований",
        "частично подходит"
    ]

    not_ideal_keywords = [
        "не соответствует",
        "не подходит",
        "существенные недостатки",
        "не выполнены основные требования",
        "не удовлетворяет критериям"
    ]

    if any(keyword in analysis_lower for keyword in ideal_keywords):
        return "Идеально подходит"

    elif any(keyword in analysis_lower for keyword in partial_keywords):
        return "Частично подходит"

    elif any(keyword in analysis_lower for keyword in not_ideal_keywords):
        return "Не подходит"

    elif any(keyword in analysis_lower for keyword in ["соответствует", "удовлетворяет", "сильные стороны", "положительные качества"]):
        return "Частично подходит"

    return "Не подходит"

# Функция для работы с интерфейсом User (проверка соответствия)
def assistant_interface_user(job_description, resume_file):
    resume_text = extract_text_from_pdf(resume_file.name)
    job_requirements = analyze_job_vacancy(job_description)
    analysis = analyze_resume_against_vacancy(resume_text, job_requirements)

    candidate_status = categorize_candidate(analysis)

    if candidate_status != "Не подходит":
        return "Вы успешно прошли на второй этап", True  # Возвращаем статус и флаг успешного прохождения
    else:
        return "К сожалению, ваше резюме не соответствует требованиям для первого этапа", False

# Функция для предоставления задания кандидату
def give_assignment_for_candidate(vacancy_description):
    prompt = f"Предоставь задание для кандидата, исходя из следующего описания вакансии: {vacancy_description}\nЗадание должно быть практическим и связано с необходимыми навыками для данной работы."
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "system", "content": "Вы ассистент, который генерирует задания для кандидатов по описанию вакансии."},
                      {"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Ошибка при создании задания: {e}"


# Функция для обработки регистрации на собеседование
def register_interview(name, phone, email, interview_time):
    # Сохраняем информацию о записи в CSV
    new_entry = {
        "Имя": name,
        "Телефон": phone,
        "Email": email,
        "Время собеседования": interview_time
    }

    try:
        df = pd.read_csv("/content/interview_schedule.csv")
        df = df.append(new_entry, ignore_index=True)
    except FileNotFoundError:
        df = pd.DataFrame([new_entry])

    df.to_csv("/content/interview_schedule.csv", index=False)

# Интерфейс Gradio
def download_csv():
    return "/content/candidate_results.csv"



def extract_text_from_pdf2(pdf_file):
    text = ""
    try:
        with open(pdf_file, "rb") as file:
            reader = PyPDF2.PdfReader(file)
            for page in reader.pages:
                text += page.extract_text() + "\n"
    except Exception as e:
        return f"Ошибка при извлечении текста из PDF: {str(e)}"
    return text

# Функция для анализа вакансии (извлечение требований)
def analyze_job_vacancy2(job_description_text):
    prompt = f"Проанализируй описание вакансии и извлеки следующие данные:\n1. Основные требования (например, опыт, навыки и образование)\n2. Должностные обязанности\n\nОписание вакансии:\n{job_description_text}"
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "system", "content": "Вы — ассистент, который анализирует вакансии и извлекает требования, опыт и другие детали."},
                      {"role": "user", "content": prompt}]
        )
        print(f"Ответ OpenAI по вакансии: {response.choices[0].message.content}")  # Отладочный вывод
        return response.choices[0].message.content
    except Exception as e:
        return f"Ошибка при обработке вакансии: {e}"

# Функция для анализа резюме
def analyze_resume_against_vacancy2(resume_text, job_requirements):
    prompt = f"Прочитай следующее резюме:\n{resume_text}\n\nСравни с требованиями вакансии:\n{job_requirements}\nОпредели, насколько резюме соответствует указанным критериям, и предоставь детальное сравнение, включая сильные и слабые стороны кандидата."
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "system", "content": "Вы — ассистент, который отбирает кандидатов на вакансии."},
                      {"role": "user", "content": prompt}]
        )
        print(f"Ответ OpenAI по резюме: {response.choices[0].message.content}")  # Отладочный вывод
        return response.choices[0].message.content
    except Exception as e:
        return f"Ошибка при анализе резюме: {str(e)}"

# Функция для работы с интерфейсом User (проверка соответствия и второй этап)
def assistant_interface_user2(job_description, resume_file):
    resume_text = extract_text_from_pdf(resume_file.name)
    job_requirements = analyze_job_vacancy(job_description)
    analysis = analyze_resume_against_vacancy(resume_text, job_requirements)

    candidate_status = categorize_candidate(analysis)

    print(f"Статус кандидата: {candidate_status}")  # Отладочный вывод

    if candidate_status != "Не подходит":
        # Генерация задачи для второго этапа через OpenAI, если резюме подходит
        task_prompt = f"На основе описания вакансии: {job_description}\nСоздайте задачу для второго этапа собеседования, связанную с этой вакансией. Несложную"
        try:
            task_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "system", "content": "Вы — ассистент, который генерирует задания для собеседований."},
                          {"role": "user", "content": task_prompt}]
            )
            task_description = task_response.choices[0].message.content
            return f"Вы успешно прошли на первый этап. Задание для второго этапа: {task_description}", task_description
        except Exception as e:
            return f"Ошибка при генерации задания для второго этапа: {str(e)}", None
    else:
        return "К сожалению, ваше резюме не соответствует требованиям для первого этапа", None


# Функция для проверки выполнения задания
def check_second_stage_answer2(task_description, user_answer, user_email=None, resume_link=None):
    prompt = f"Задание:\n{task_description}\n\nОтвет пользователя:\n{user_answer}\n\nПроверьте, соответствует ли ответ требованиям задания, и дайте краткую оценку, включая правильность выполнения."
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "system", "content": "Вы — ассистент, который проверяет правильность выполнения заданий."},
                      {"role": "user", "content": prompt}]
        )
        evaluation = response.choices[0].message.content

        print(f"Оценка выполнения задания: {evaluation}")  # Отладочный вывод

        # Оценка выполнения задания
        test_result = "Не прошел"
        if "правильно" in evaluation.lower() or "удовлетворительно" in evaluation.lower() or "Задание выполнено верно" in evaluation.lower() or "полностью соответствует требованиям" in evaluation.lower():
            test_result = "Пройден"

        # Формируем данные для записи в CSV
        result_data = {
            "Задание второго этапа": task_description,
            "Ответ пользователя": user_answer,
            "Оценка выполнения": evaluation,
            "Результат теста": test_result,
            "Email": user_email if user_email else "Не указан",  # Если email не передан, ставим "Не указан"
            "Ссылка на резюме": resume_link if resume_link else "Не указана"  # Если ссылка на резюме не передана, ставим "Не указана"
        }

        # Сохраняем результат в CSV
        save_to_csv(result_data, "/content/second_stage_results.csv")

        if test_result == "Пройден":
            return f"Ваш ответ проверен. Вы приглашены на собеседование! Оценка: {evaluation}"
        else:
            return f"Ваш ответ проверен. К сожалению, вы не прошли дальше. Оценка: {evaluation}"
    except Exception as e:
        return f"Ошибка при проверке задания: {str(e)}"


with gr.Blocks() as iface:
    with gr.TabItem("Результаты анализа HR"):
        gr.Markdown("### Результаты анализа резюме")
        job_description = gr.Textbox(label="Описание вакансии", lines=5, placeholder="Введите описание вакансии...")
        resume_files = gr.File(label="Загрузите резюме (PDF)", file_count="multiple", type="filepath")
        output_results = gr.DataFrame(label="Результаты анализа")

        submit_results = gr.Button("Получить результаты")
        submit_results.click(assistant_interface_results, [job_description, resume_files], output_results)

        download_button = gr.Button("Скачать результаты")


    with gr.TabItem("Проверка для пользователя"):
        gr.Markdown("### Проверка соответствия резюме вакансии")
        job_description_user2 = gr.Textbox(label="Описание вакансии", lines=5, placeholder="Введите описание вакансии...")
        resume_file_user2 = gr.File(label="Загрузите ваше резюме (PDF)", type="filepath")
        user_email_input = gr.Textbox(label="Введите ваш email", placeholder="example@domain.com")  # Поле для email
        output_user2 = gr.Textbox(label="Результат проверки")
        task_output2 = gr.Textbox(label="Задание для второго этапа", interactive=False)

        submit_user2 = gr.Button("Проверить соответствие")
        submit_user2.click(assistant_interface_user2, [job_description_user2, resume_file_user2], [output_user2, task_output2])

        gr.Markdown("### Ответ на задание второго этапа")
        user_task_response2 = gr.Textbox(label="Ваш ответ на задание", lines=5, placeholder="Введите ваш ответ...")
        output_second_stage2 = gr.Textbox(label="Результат проверки")

        submit_task2 = gr.Button("Отправить ответ")
        submit_task2.click(check_second_stage_answer2, [task_output2, user_task_response2, user_email_input], output_second_stage2)

        # Кнопка для скачивания результата теста
        download_test_result_button = gr.Button("Скачать результаты теста")
        download_test_result_button.click(download_csv, [], None)  # Для скачивания CSV с результатами тестов
        download_button.click(download_csv, [], None)  # Для скачивания CSV

iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4fd808686f295e90d5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
